<a href="https://colab.research.google.com/github/MHosseinHashemi/Image_Similarity/blob/main/Image_Simmilarity_CenterLoss_TF_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, RandomFlip, RandomRotation, Dense, Dropout, Lambda

from tqdm import tqdm
from collections import defaultdict

In [3]:
(train_data, test_data, validation_data), info = tfds.load("oxford_flowers102", split=['train', 'validation', 'test'], as_supervised=True, with_info=True)

In [4]:
height = 128
width = 128

def preprocess_images(image, label, height, width):
    # image = tf.image.resize_with_crop_or_pad(image, target_height=height, target_width=width)
    image = tf.image.resize(image, [width, height])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [5]:
train_ds = train_data.map(lambda image, label: preprocess_images(image, label, height, width))

In [6]:
test_ds = test_data.map(lambda image, label: preprocess_images(image, label, height, width))

In [7]:
def data_loader(data):
  x = []
  y = []
  for img, label in tqdm(data.as_numpy_iterator()):
    x.append(img)
    y.append(label)

  return x, y

In [8]:
x_train, y_train = data_loader(train_ds)

1020it [00:02, 451.41it/s]


In [9]:
x_test, y_test = data_loader(test_ds)

1020it [00:02, 458.73it/s]


In [10]:
# Base Model
MODEL_URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2"

# model = tf.keras.Sequential([
#     tf.keras.layers.RandomFlip(),
#     tf.keras.layers.RandomRotation(0.3),
#     hub.KerasLayer(MODEL_URL, trainable=True),
#     tf.keras.layers.Dropout(0.25),
#     tf.keras.layers.Dense(128, activation=None),
#     tf.keras.layers.Dense(102, activation='softmax')
#     # tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
# ])

# model.build([None, height, height, 3])
# model.summary()

# With Functional API to prevent Further Err
input_layer = Input(shape=(height, width, 3))
x = RandomFlip()(input_layer)
x = RandomRotation(0.3)(x)
x = hub.KerasLayer(MODEL_URL, trainable=True)(x)
x = Dropout(0.25)(x)
x = Dense(128, activation=None)(x)
x = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)  # L2 normalize embeddings
output_layer = Dense(102, activation='softmax')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 random_flip (RandomFlip)    (None, 128, 128, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 128, 128, 3)      0         
 tion)                                                           
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              20331360  
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                             

In [11]:
def batch_me(images, labels, batch_size, samples_per_class):
  temp_dict = defaultdict(list) # A Dic of Lists to save img, label pairs as one object
  for img, label in zip(images, labels):
    temp_dict[label].append(img)

  while True:
    batch_x = []
    batch_y = []
    while len(batch_x) < batch_size:
      for category, examples in temp_dict.items():
        # Only feed as large as the "samples per class"
        # If the batch did not had enough space, feed as much as it has
        n_samples = min(samples_per_class, (batch_size - len(batch_x)))
        if n_samples == 0:
          break
        # Pick randomly from simmilar images of the same category
        samples = random.sample(examples, k=n_samples)
        # Add corresponding x, y values to the batch
        batch_x.extend(samples)
        batch_y.extend([category] * len(samples))

    # It should be a continous operation
    yield np.array(batch_x), np.array(batch_y)


In [12]:
def center_loss(feature_vector, center):
    difference = feature_vector - center
    loss = tf.reduce_mean(tf.reduce_sum(difference**2, axis=1))

    return loss

# Debug

In [13]:
# x_train[0].shape

In [14]:
# temp=np.expand_dims(x_train[0], axis=0)
# temp.shape

In [19]:
# feature_extraction_model = Model(inputs=model.input, outputs=model.layers[-2].output)

In [16]:
# feature_extraction_model.predict(temp).shape

In [17]:
# l_1 = [21, 12, 33, 46]
# l_2 = [15, 16, 17, 18]

# result_dict = {key: value for key, value in zip(l_1, l_2)}

# print(result_dict)

# End of Debug

In [20]:
feature_extraction_model = Model(inputs=model.input, outputs=model.layers[-2].output)

In [17]:
# # Extarct All feature vectors and their corresponding centers
# raw_features = {}
# for k in tqdm(range(len(x_train))):
#     if y_train[k] in raw_features.keys():
#         # Avergae the new center with the previous one and replace it
#         new_center = feature_extraction_model.predict(np.expand_dims(x_train[k], axis=0)).mean()
#         prev_center = raw_features[y_train[k]]
#         raw_features[y_train[k]] = [(prev_center + new_center )/2] * 128
#         del new_center, prev_center

#     else:
#         raw_features[y_train[k]].append([feature_extraction_model.predict(np.expand_dims(x_train[k], axis=0)).mean()] * 128)


# all_features = {key: value for key, value in raw_features.items()}

# del raw_features,
# all_features

In [ ]:
# raw_features = {}

# for k, (x, y) in tqdm(enumerate(zip(x_train, y_train))):
#     if y in raw_features:
#         new_center = feature_extraction_model.predict(np.expand_dims(x, axis=0)).mean()
#         prev_center = raw_features[y]
#         averaged_center = [(prev + new_center) / 2 for prev in prev_center]
#         raw_features[y] = [averaged_center] * 128
#     else:
#         feature_vector = feature_extraction_model.predict(np.expand_dims(x, axis=0)).mean()
#         raw_features[y] = [feature_vector] * 128

# all_features = {key: value for key, value in raw_features.items()}

In [21]:
raw_features = {}
class_feature_vectors = {}  # Store pre-calculated feature vectors for each class

# Calculate and store class feature vectors
for x, y in tqdm(zip(x_train, y_train)):
    if y not in class_feature_vectors:
        feature_vector = feature_extraction_model.predict(np.expand_dims(x, axis=0)).mean()
        class_feature_vectors[y] = feature_vector

# Calculate raw_features using class_feature_vectors
for x, y in tqdm(zip(x_train, y_train)):
    if y in raw_features:
        new_center = feature_extraction_model.predict(np.expand_dims(x, axis=0)).mean()
        raw_features[y] = [(prev + new_center) / 2 for prev in raw_features[y]]
    else:
        raw_features[y] = [class_feature_vectors[y]] * 128

all_features = {key: value for key, value in raw_features.items()}

# Clean up memory by releasing unnecessary variables
del class_feature_vectors


0it [00:00, ?it/s]

1/1 [==============================] - 3s 3s/step


1it [00:02,  2.93s/it]

1/1 [==============================] - 0s 110ms/step


2it [00:03,  1.30s/it]

...

1018it [02:37,  6.20it/s]

1/1 [==============================] - 0s 102ms/step


1019it [02:37,  6.06it/s]

1/1 [==============================] - 0s 96ms/step


1020it [02:37,  6.46it/s]


In [22]:
all_features

{72: [-0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.005025452923291596,
  -0.0050254529232915

In [23]:
epochs = 50
alpha = 0.5
batch_size = 16
n_examples_per_class = 4
EMA_lr = 0.9
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)


# Calculate initial centers
centers = tf.Variable(initial_value=tf.random.normal((102, 128), mean=0.0, stddev=0.5))

# Loop through all samples

for index in range(102):
    # category_features = all_features[y_train == index]
    category_center = all_features[y_train == index]
    # if len(category_features) > 0:
    if len(category_center) > 0 :
        # category_center = tf.reduce_mean(category_features, axis=0)
        centers[index].assign(category_center)

# Training
for epoch in tqdm(range(epochs)):
    total_loss = 0.0
    num_batches = 0
    for batch_x, batch_y in batch_me(images=x_train, labels=y_train, batch_size=batch_size, samples_per_class=n_examples_per_class):
        # Capture Gradients
        with tf.GradientTape() as tape:
            # Extract Features per batch
            temps = model(batch_x, training=False)
            # Seprate labels and their features
            predictions = temps[0]
            features = temps[1]
            # initialize batch centers
            batch_centers = centers.numpy()[batch_y]
            # Calculate Batch Centers
            for index in range(batch_size):
                instance_features = features[index]
                instance_mean = tf.reduce_mean(instance_features, axis=0)
                batch_centers[index] = instance_mean

            # Center-Loss calculation
            c_loss = center_loss(features, batch_centers)
            # Combine it with CategoricalCrossEntropyLoss
            cls_loss = tf.keras.losses.CategoricalCrossentropy()(batch_y, predictions)
            # Total Loss
            loss = (c_loss * alpha) + cls_loss

        # Calculate Gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Update training loss
        total_loss += loss
        num_batches += 1

    # Calculate training Loss
    training_loss = total_loss / num_batches


    """Validation"""
    val_temps = model(x_test)
    # Extract Validation Features and Labels
    val_predictions = val_temps[0]
    val_features = val_temps[1]

    # Center Calcualtion
    val_batch_centers = centers.numpy()[y_test]
    for idx in range(len(y_test)):
        val_instance_features = val_features[idx]
        val_instance_mean = tf.reduce_mean(val_instance_features, axis=0)
        val_batch_centers[idx] = val_instance_mean

    # Loss Calculation
    val_c_loss = center_loss(val_features, val_batch_centers)
    val_cls_loss = tf.keras.losses.CategoricalCrossentropy()(y_test, val_predictions)
    val_loss = (val_c_loss * alpha) + val_cls_loss

    print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {training_loss:.4f} - Validation Loss: {val_loss.numpy():.4f}")

    # Centers Update Frequency
    for index in range(102):
        category_features = all_features[y_train == index]
        if len(category_features)>0:
            category_center = tf.reduce_mean(category_features, axis=0)
            centers[index].assign((1.0 - EMA_lr) * centers[index]) + (EMA_lr * category_center)
            print(f"Centers updated - Step : {epoch+1}\n")


  0%|          | 0/50 [00:14<?, ?it/s]


InvalidArgumentError: ignored

# **TODO**

*The provided code seems to be an attempt to train a model for image similarity using both center loss and categorical cross-entropy loss on the Oxford 102 flowers dataset. Overall, the code structure and approach are reasonable, but there are a few areas that could be improved or clarified*:


- **Center Update**: The code updates the center values using the centers variable, but it initializes this variable with a random normal distribution. It might be more effective to initialize the centers with zeros, as this provides a better starting point.

- **Center Calculation**: The code calculates the batch centers for each batch using the mean of the feature vectors. However, in this implementation, the same center values are updated for each category within the batch. It's more common to calculate centers per category over the entire training set and then update them as the training progresses.

- **Learning Rate Schedule**: Consider using a learning rate schedule, such as the tf.keras.optimizers.schedules module, to adjust the learning rate during training. This can help with convergence and stability.

- **Batch Size and Center Calculation**: The current code calculates centers based on batch_y, which means it uses the current batch for calculating centers. This might lead to unstable center values. It's better to accumulate the feature vectors for each category over the entire training set and then calculate centers after an epoch.

- **Batch Normalization**: The model uses a pretrained EfficientNet model, which is a good choice. However, consider using batch normalization layers after the hub layer to improve training stability.

- **Variable Names**: Some variable names could be more descriptive. For example, temps and val_temps might be better named as outputs or something similar.

- **Validation Loop**: The validation loop calculates the loss for the entire validation set but doesn't update any values. It's common to calculate metrics such as accuracy as well.

- **Center Loss Term**: The alpha parameter is used to weight the center loss term. You might need to experiment with this value to achieve a good balance between the two loss components.

- **Normalization**: While the preprocessing of images includes scaling them to the range [0, 1], you may also consider standardizing the images (subtracting mean and dividing by standard deviation) to help convergence.

- **Model Checkpoints**: Consider using model checkpoints to save the best model during training based on validation performance.